In [1]:
import cv2
import numpy as np

# Загрузка видеопоследовательности
cap = cv2.VideoCapture('videoplayback.mp4')

# Инициализация параметров для алгоритма Лукаса-Канаде
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Чтение первого кадра
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Определение начальных точек для отслеживания (можно изменить)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7)

# Создание массива для отслеживания точек
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Вычисление оптического потока с использованием алгоритма Лукаса-Канаде
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Отбор хороших точек
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Рисование линий между точками
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        x1, y1 = new.ravel()
        x2, y2 = old.ravel()
        mask = cv2.line(mask, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(x1), int(y1)), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)

    cv2.imshow('Optical Flow', img)
    if cv2.waitKey(30) & 0xFF == 27:
        break

    # Обновление кадра и точек
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 